# About: Moodle環境の削除--Azure仮想マシンの削除

---

Azureの仮想マシンを削除します。

## パラメータの設定

仮想マシンを削除するのに必要となるパラメータを設定します。

### Ansibleのグループを指定する

削除対象の仮想マシンがあるAnsibleのグループ名を指定してください。

既存のグループ名を確認するために`group_vars`にあるディレクトリの一覧を表示します。

In [ ]:
!ls -1 group_vars/

上のセルの出力結果を参考にしてAnsibleのグループ名を次のセルに指定してください。

In [ ]:
# (例)
# target_group = 'Moodle'

target_group = 

#### チェック

指定された `target_group` の値が適切なものかチェックします。

`target_group` に対応する設定ファイルが存在していることを確認します。

In [ ]:
from pathlib import Path

target_dir = Path('group_vars') / target_group
if not (target_dir.is_dir() and len(list(target_dir.glob("*"))) > 0):
    target_file = Path('group_vars') / f"{target_group}.yml"
    if target_file.is_file():
        target_dir.mkdir(exist_ok=True)
        target_file.rename(target_dir / "00-moodle.yml")
    else:
        raise RuntimeError(f"ERROR: not exists {target_group}")

### Azureの認証情報の設定

Azureの操作を行うために必要とサービスプリンシパルに関する情報を指定します。

サブスクリプションIDの値を入力してください。

In [ ]:
from getpass import getpass
azure_subscription_id = getpass()

テナントIDを入力してください。

In [ ]:
azure_tenant = getpass()

クライアントIDを入力してください。

In [ ]:
azure_client_id = getpass()

シークレットを入力してください。

In [ ]:
azure_secret = getpass()

このNotebookから実行するコマンドでAzureのサービスプリンシパルを参照できるようにするために、[環境変数(`AZURE_SUBSCRIPTION_ID`, `AZURE_TENANT`, `AZURE_CLIENT_ID`, `AZURE_SECRET`)](https://docs.microsoft.com/ja-jp/azure/developer/ansible/install-on-linux-vm#env-credentials)の設定を行います。

In [ ]:
import os

os.environ['AZURE_SUBSCRIPTION_ID'] = azure_subscription_id
os.environ['AZURE_TENANT'] = azure_tenant
os.environ['AZURE_CLIENT_ID'] = azure_client_id
os.environ['AZURE_SECRET'] = azure_secret

### 削除対象となる仮想マシンの指定

削除対象となる仮想マシンの名前を指定します。

`group_vars`ファイルに記録されている仮想マシンの名前を表示します。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(target_group)
vm_name = gvars['azure_vm_name']
print(vm_name)

`group_vars`ファイルに記録されているリソースグループの値を表示します。

In [ ]:
resource_group = gvars['azure_resource_group']
print(resource_group)

## 仮想マシンの削除

実際にAzureの仮想マシンを削除します。

In [ ]:
!.venv/bin/ansible localhost -c local -m azure.azcollection.azure_rm_virtualmachine -a \
    'state=absent resource_group={resource_group} name={vm_name} \
    remove_on_absent=["all_autocreated"]'

## Ansible 設定ファイルの後始末

Ansibleに関する設定ファイルの後始末を行います。

group_vars に記録されているパラメータを無効にするために group_vars ファイルをリネームします。

In [ ]:
!mv group_vars/{target_group}  group_vars/.{target_group}.$(date +"%Y%m%d%H%M%S").bak

インベントリから対象となるグループを削除します。

In [ ]:
%run scripts/group.py
remove_group_from_inventory_yml(target_group)

# 削除前との差分を表示します。
!diff -u inventory.yml.bak inventory.yml || true